##Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib. pyplot as plt
import seaborn as sns

## Load the dataset in Python environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Datasets/mobile_price_classification.csv')

## Data preprocessing

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   battery_power   2000 non-null   int64  
 1   bluetooth       2000 non-null   int64  
 2   clock_speed     2000 non-null   float64
 3   dual_sim        2000 non-null   int64  
 4   front_cam       2000 non-null   int64  
 5   4G              2000 non-null   int64  
 6   int_memory      2000 non-null   int64  
 7   m_dep           2000 non-null   float64
 8   mobile_wt       2000 non-null   int64  
 9   n_cores         2000 non-null   int64  
 10  primary_camera  2000 non-null   int64  
 11  px_height       2000 non-null   int64  
 12  px_width        2000 non-null   int64  
 13  ram             2000 non-null   int64  
 14  sc_h            2000 non-null   int64  
 15  sc_w            2000 non-null   int64  
 16  talk_time       2000 non-null   int64  
 17  three_g         2000 non-null   i

In [ ]:
data.head()

,battery_power,bluetooth,clock_speed,dual_sim,front_cam,4G,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [ ]:
#check for null values
data.isna().sum()

,0
battery_power,0
bluetooth,0
clock_speed,0
dual_sim,0
front_cam,0
4G,0
int_memory,0
m_dep,0
mobile_wt,0
n_cores,0


In [ ]:
#checking dor duplicates
data.duplicated().sum()

0

In [ ]:
# check the target column
data['price_range'].value_counts()

,count
price_range,
1,500
2,500
3,500
0,500


In our data, there doesnot exist any null values and duplicates. All the features are numerical and no need of encoding, so we directly move to ANN

In [ ]:
#Feature target splitting and split train test
x=data.drop('price_range', axis=1)
y=data['price_range']


from sklearn.model_selection import train_test_split
x_train,x_test,y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [ ]:
#scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)


In [ ]:
# label encoding to y variable-4 price ranges
from keras.utils import to_categorical

y_train_cat = to_categorical(y_train, num_classes=4)  # There are 4 price ranges: 0, 1, 2, 3
y_test_cat = to_categorical(y_test, num_classes=4)

In [ ]:
x_train.shape

(1600, 20)

In [ ]:
x_test.shape

(400, 20)

In [ ]:
y_train.shape

(1600,)

In [ ]:
y_test.shape

(400,)

##ANN model- without doing hyperparameter tuning

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
#initialize the mpdel:create a sequential model where layers can be added one after anaother
classifier=Sequential()

In [ ]:
#Build the model

#Adding the input layer and the first hidden layer#10 neurons in first layer each with relu activation function, input dim is 20 features input)
classifier.add(Dense(units=10, kernel_initializer ='uniform', activation='relu',input_dim = 20))

#Adding the second hidden layer
classifier.add(Dense(units=10, kernel_initializer ='uniform', activation='relu'))

#Adding  # Output layer with 4 classes
classifier.add(Dense(units=4, kernel_initializer ='uniform', activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model using the adam optimizer with categorical cross entropy
classifier.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
classifier.fit(x_train, y_train_cat, epochs=50, batch_size=20, verbose=0)


In [ ]:
## Evaluate the model
y_pred = np.argmax(classifier.predict(x_test), axis=1)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [ ]:
# Convert y_test_categorical back to multiclass labels
y_test_multiclass = np.argmax(y_test_cat, axis=1)

In [ ]:
# Evaluate the model
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(y_test_multiclass, y_pred)
print(cm)

accuracy = accuracy_score(y_test_multiclass, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test_multiclass, y_pred))

[[103   2   0   0]
 [  3  86   2   0]
 [  0   3  83   6]
 [  0   0   0 112]]
Accuracy: 0.96
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.98      0.98       105
           1       0.95      0.95      0.95        91
           2       0.98      0.90      0.94        92
           3       0.95      1.00      0.97       112

    accuracy                           0.96       400
   macro avg       0.96      0.96      0.96       400
weighted avg       0.96      0.96      0.96       400



In [ ]:
# evaluate our test model
score, acc =classifier.evaluate(x_test,y_test_cat,verbose =0)
print('Test score:', score)
print('Test acc:', acc)

Test score: 0.12010917067527771
Test acc: 0.9599999785423279


In [ ]:
classifier.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 10)                  │             210 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             110 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 4)                   │              44 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,094 (4.28 KB)

 Trainable params: 364 (1.42 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 730 (2.86 KB)

##Hyperparameter tuning for ANN model

In [ ]:
df =pd.read_csv('/content/drive/MyDrive/Datasets/mobile_price_classification.csv')

##load necessary packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import keras_tuner as kt
import tensorflow as tf




In [ ]:
# Separate features and target
X = data.drop('price_range', axis=1)
y = data['price_range']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)



In [ ]:
# Convert target variable to categorical
y_train_cat = to_categorical(y_train, num_classes=4)
y_test_cat= to_categorical(y_test, num_classes=4)


In [ ]:
X_train.shape[1]

20

In [ ]:

# Define a model-building function for Keras Tuner
def build_model(hp):
    model = Sequential()
    # Input layer
    model.add(Dense(hp.Int('units_1', min_value=32, max_value=128, step=16), activation='relu', input_dim=X_train.shape[1]))


    # Hidden layers
    for i in range(hp.Int('num_hidden_layers', 1, 3)):
        model.add(Dense(hp.Int(f'units_{i+2}', min_value=32, max_value=128, step=16), activation='relu'))


    # Output layer
    model.add(Dense(4, activation='softmax'))  # 4 classes for price range

    # Compile the model
    model.compile(
      optimizer = tf.keras.optimizers.get({
          'class_name': hp.Choice('optimizer',values=['adam','rmsprop','sgd']),
          'config' : {'learning_rate':hp.Float('learning_rate',min_value=1e-4,max_value=1e-2,sampling= 'log')}
      }),
      loss='categorical_crossentropy',
      metrics=['accuracy']
      )
    return model



In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 5
)

In [ ]:
# Search for the best hyperparameters
tuner.search(X_train, y_train_cat, validation_data=(X_test, y_test_cat), epochs=20, batch_size=32)



Trial 5 Complete [00h 00m 07s]
val_accuracy: 0.32749998569488525

Best val_accuracy So Far: 0.9375
Total elapsed time: 00h 00m 39s


In [ ]:

# Retrieve the best model and hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters:", best_hps.values)



Best hyperparameters: {'units_1': 112, 'num_hidden_layers': 1, 'units_2': 112, 'optimizer': 'rmsprop', 'learning_rate': 0.0022080982419546116}


In [ ]:
# Train the best model
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train, y_train_cat, validation_data=(X_test, y_test_cat), epochs=50, batch_size=32)



Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5112 - loss: 1.0732 - val_accuracy: 0.8125 - val_loss: 0.4943
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8865 - loss: 0.3989 - val_accuracy: 0.8875 - val_loss: 0.3068
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9293 - loss: 0.2227 - val_accuracy: 0.9100 - val_loss: 0.2364
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9401 - loss: 0.1732 - val_accuracy: 0.8975 - val_loss: 0.2209
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9581 - loss: 0.1207 - val_accuracy: 0.9050 - val_loss: 0.2295
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9623 - loss: 0.0990 - val_accuracy: 0.9100 - val_loss: 0.2152
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9734 - loss: 0.0784 - val_accuracy: 0.9275 - val_loss: 0.2007
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9819 - loss: 0.0580 - val_accuracy: 0.9225 - val_loss:

In [ ]:
# Evaluate the model
accuracy = best_model.evaluate(X_test, y_test_cat, verbose=0)[1]
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 92.25%
